# CS 4964 - Final Project



https://huggingface.co/datasets/speech_commands/viewer


In [3]:
from speech_commands.speech_commands import SpeechCommands

ModuleNotFoundError: No module named 'datasets'

In [1]:
speechCommands = SpeechCommands()afsasfas


wtf is this

SyntaxError: invalid syntax (927626257.py, line 1)